In [2]:
from data_cleaning import Open, Stream, MakeMIDI, ChordEval
from music21 import chord, note,stream
import os

In [ ]:
mid_file = os.path.join(os.getcwd(), "simple_cmajor.mid")
midi_data = Open.open_midi(mid_file)